In [1]:
%env XLA_IR_DEBUG=1
%env XLA_HLO_DEBUG=1

env: XLA_IR_DEBUG=1
env: XLA_HLO_DEBUG=1


In [2]:
import torch
import torch.nn as nn
import torch_xla

In [3]:
from torch_xla.experimental.scan import scan

In [6]:
def step_fn(carry, x):
  new_carry = carry + x
  y = new_carry + torch.rand(2, device=torch_xla.device())
  return new_carry, y

init = torch.tensor([0.0, 0.0], device=torch_xla.device())
xs = torch.tensor([[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], device=torch_xla.device())

# Bug: because we reuse the same graph from the first layer, all the random numbers
# produced from each step are the same.
#
# To fix the bug, we need to verify that the HLO from each step are the same,
# and that any additional input tensors (not part of `xs`) must also be the same.
# Then the use of RNG will result in different HLOs and at least we can fail with
# an error.
#
# In contrast, in JAX and Flax, people thread the RNG state explicitly, and they
# can simply fork the RNG before calling each decoder layer. They can even pre-fork
# the RNG state and give a bunch of different seeds to `scan`.
scan(step_fn, init, xs)

GetRngSeed of device 0x560b0db483c0
Init seed value from scalar: 101
Update seed to: 24146324


(tensor([0., 0.], device='xla:0'),
 tensor([[0.4245, 0.1542],
         [0.4245, 0.1542],
         [0.4245, 0.1542]], device='xla:0'))